In [3]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
NAFCUSTOMER_C360_ACCOUNTS = dataiku.Dataset("NAFCUSTOMER_C360_ACCOUNTS")
NAFCUSTOMER_C360_ACCOUNTS_df = NAFCUSTOMER_C360_ACCOUNTS.get_dataframe()
print(len(NAFCUSTOMER_C360_ACCOUNTS_df))

NAFCUSTOMER_REVENUE_BY_CUSTOMER_LIMITED = dataiku.Dataset("NAFCUSTOMER_REVENUE_BY_CUSTOMER_LIMITED")
NAFCUSTOMER_REVENUE_BY_CUSTOMER_LIMITED_df = NAFCUSTOMER_REVENUE_BY_CUSTOMER_LIMITED.get_dataframe()
print(len(NAFCUSTOMER_REVENUE_BY_CUSTOMER_LIMITED_df))

844417
21434


In [4]:
NAFCUSTOMER_REVENUE_BY_CUSTOMER_LIMITED_df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,CUSTOMER,ACCOUNTSTATUS,SETUP_DATE,ACCOUNT_CLOSED_DATE,REVENUE_YEAR,REVENUE_QUARTER,REVENUE_MONTH,CUSTOMER_SOURCE_SYSTEM_CODE,CUSTOMER_BUSINESS_PROGRAM_NAME,BI_CUS_PAR,BI_PRODUCT,BI_MRU,REVENUE_AMOUNT_USD,GROSS_SPEND_AMOUNT,PURCHASE_GALLONS_QTY
0,469008444325,ENDO PHARMACEUTICALS (EPH),ENDO PHARMACEUTICALS (EPH),Active,2011-03-07 08:00:00+00:00,9999-12-31 00:00:00.000,2019,1,1,TANDEM,Wheels old,33200,10200,3100,411.541966,21032.23,8632.216
1,469005700778,RAYTHEON COMPANY (0469),RAYTHEON COMPANY (0469),Terminated,2016-05-24 07:00:00+00:00,2019-05-01 00:00:00.000,2019,1,1,TANDEM,Wheels old,33200,10200,3100,14.760585,571.05,231.016
2,469005701925,LABCORP (3LAB),LABORATORY CORPORATION OF AMERICA,Terminated,2018-10-03 07:00:00+00:00,2021-12-07 00:00:00.000,2019,1,1,TANDEM,Wheels old,33200,10200,3100,27725.567526,1640316.24,702892.287
3,469007590169,PB,BALFOUR PB,Terminated,2007-10-12 07:00:00+00:00,2022-07-12 00:00:00.000,2019,1,1,TANDEM,Wheels old,33200,10200,3100,2225.512767,113508.92,46724.697
4,469008711962,"DAIICHI SANKYO, INC. (2)","DAIICHI SANKYO, INC.",Terminated,2012-03-19 07:00:00+00:00,2020-05-29 00:00:00.000,2019,1,1,TANDEM,Wheels old,33200,10200,3100,1721.795202,100059.56,42614.566


In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.



CALCULATED_DRAW_DOWNS_df = NAFCUSTOMER_C360_ACCOUNTS_df # For this sample code, simply copy input to output


# Write recipe outputs
CALCULATED_DRAW_DOWNS = dataiku.Dataset("CALCULATED_DRAW_DOWNS")
CALCULATED_DRAW_DOWNS.write_with_schema(CALCULATED_DRAW_DOWNS_df)